# Resources

https://www.s-anand.net/blog/automating-powerpoint-with-python/

https://ericgarlic.blog/2018/03/24/how-to-generate-ms-office-constants/

https://ericgarlic.blog/category/win32com/

usefull
http://timgolden.me.uk/python/win32_how_do_i/generate-a-static-com-proxy.html

http://msdn.microsoft.com/en-us/library/ff604967%28office.14%29.aspx

https://nbviewer.ipython.org/github/sanand0/ipython-notebooks/blob/master/Office.ipynb

In [1]:
import sys 
import win32com.client as win32
from pathlib import Path

print(sys.version)

# cria .py com constantes de referencia
# constants = [
#     ("Microsoft Office 16.0 Object Library", "./MSOconstants/MSO.py"),
#     #("Microsoft Excel 16.0 Object Library","./MSOconstants/MSXLS.py"),
#     (r"C:\Program Files\Microsoft Office\root\Office16\MSPPT.OLB", "./MSOconstants/MSPPT.py"),
#     #("Microsoft Word 16.0 Object Library","./MSOconstants/MSWORD.py"),
# ]
# for officeApp in constants:
#     print("Processing ",officeApp[0])
#     sys.argv = ['', "-o", officeApp[1], officeApp[0]]
#     win32.makepy.main()
    
    
def create_modules(reference: [tuple, str]):
    """
    create .py with constants and enumerations of reference library. 
    - reference = (reference name, new module path) or reference library file path.
    """
    if isinstance(reference, tuple):
        from_path, to_path = reference
    elif isinstance(reference, str):
        from_path = Path(reference)
        assert from_path.exists()
        assert from_path.is_file()
        to_path = f"./MSOconstants/{from_path.stem}.py"
    else:
        raise ValueError
    
    print("Processing ",from_path, to_path)
    sys.argv = ['', "-o", str(to_path),str(from_path)]
    return win32.makepy.main()

if not Path("./MSOconstants/MSO.py").exists():
    create_modules(("Microsoft Office 16.0 Object Library", "./MSOconstants/MSO.py"))
if not Path("./MSOconstants/MSPPT.py").exists():
    create_modules(r"C:\Program Files\Microsoft Office\root\Office16\MSPPT.OLB")

# importa modulos criados
from MSOconstants.MSO import constants as MSO
from MSOconstants.MSPPT import constants as MSPPT
MSPPT.ppLayoutBlank, MSO.msoFalse, MSO.msoTrue
#win32.constants.ppLayoutBlank # sem autocomplete ....

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


(12, 0, -1)

In [2]:
app = win32.gencache.EnsureDispatch("PowerPoint.Application")
#app = win32.Dispatch("PowerPoint.Application")

In [3]:
# abre arquivo
ppt = app.Presentations.Open(r'C:\Users\rubens\PROJETOS\py-pptx\Presentation1.pptx')

In [4]:
# primeiro slide
slide = ppt.Slides(1)#.Duplicate()

In [31]:
#https://docs.microsoft.com/en-us/office/vba/api/powerpoint.shape
sh_tpt = slide.Shapes('prod_group')

print(f'{sh_tpt.Top = }, {sh_tpt.Left = },\n{sh_tpt.Width = }, {sh_tpt.Height = }')

sh_tpt.Top = 65.91551208496094, sh_tpt.Left = 45.63378143310547,
sh_tpt.Width = 819.3803100585938, sh_tpt.Height = 150.80377197265625


In [79]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'titulo': list('ABC'),
    'texto': ["""Lorem ipsum dolor <b>sit amet</b>, consectetuer adipiscing elit. Maecenas porttitor congue massa. Fusce posuere, magna sed pulvinar ultricies, purus lectus malesuada libero, sit amet commodo magna eros quis urna.
Nunc viverra imperdiet enim. Fusce est. Vivamus a tellus. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Proin pharetra nonummy pede. Mauris et orci."""]*3,
    'aberto': [1,1,0]
})
df['texto'] = df['titulo'] + ' ' + df['texto']
df

,titulo,texto,aberto
0,A,"A Lorem ipsum dolor <b>sit amet</b>, consectet...",1
1,B,"B Lorem ipsum dolor <b>sit amet</b>, consectet...",1
2,C,"C Lorem ipsum dolor <b>sit amet</b>, consectet...",0


In [13]:
# #pd.read_excel('teste.xlsx')
# import styleframe as sf

# df2  = sf.StyleFrame.read_excel('teste.xlsx')
# df2.data_df

In [82]:
sh_prev = sh_tpt

for i, s in df.iterrows():
    
    sh = sh_tpt.Duplicate()
    sh.Left = sh_tpt.Left
    
    # atualiza texto, titulo e cadeado
    titulo = sh.GroupItems("titulo")
    titulo.TextFrame.TextRange.Text = s['titulo']
    
    texto = sh.GroupItems("texto")
    texto.TextFrame.TextRange.Text = s['texto']
    texto.TextFrame.TextRange.Characters(5,20).Font.Bold = True
    
    cadeado = sh.GroupItems("cadeado")
    cadeado.Visible = not s['aberto']
    
    # depois de colocar o texto, ajusta a posição vertical
    if sh_prev is sh_tpt:
        sh.Top = sh_prev.Top
    else:
        sh.Top = sh_prev.Top + sh.Height
    
    sh_prev = sh
    
sh_tpt.Delete()

In [76]:
sh_prev = sh_tpt
for i, s in df.iterrows():
    
    sh = sh_prev if i==0 else sh_tpt.Duplicate()
    
    sh.Left = sh_prev.Left
    sh.Top = sh_prev.Top
    
    # atualiza texto, titulo e cadeado
    titulo = sh.GroupItems("titulo")
    titulo.TextFrame.TextRange.Text = s['titulo']
    
    texto = sh.GroupItems("texto")
    texto.TextFrame.TextRange.Text = s['texto']
    
    cadeado = sh.GroupItems("cadeado")
    cadeado.Visible = not s['aberto']
    
    # depois de colocar o texto, ajusta a posição vertical
    if i>0:
        sh.Top +=  sh.Height
    
    sh_prev = sh
    
    time.sleep(1)
#sh_tpt.Delete()

In [60]:
# soma da shape1.top + altura das outras shapes
np.array([sh.Height for sh in slide.Shapes])
np.array([sh.Top for sh in slide.Shapes])
#ppt.PageSetup.SlideHeight = 

In [126]:
s= 'ab<b>def<b>bbbbbb<b/>A<b/>ghi<b>aaaaa<b/>'

In [127]:
s

'ab<b>def<b>bbbbbb<b/>A<b/>ghi<b>aaaaa<b/>'

In [124]:
def find_tag(s, tag = 'b'):
    s2 = s
    begin =  f'<{tag}>'
    end =  f'<{tag}/>'
    
    # encontra a posição das tags
    # remove as tags
    # a posicao inicial nao muda
    # a posição final -= len(tag_inicial)
    
    # encontra a 1a endtag
    p_end = s2.find(end)
    if p_end==-1: return None
    s2 = s2.replace(end, '', count=1)
    # encontra a 1a endtag
    p_begin = s2[:p_end].rfind(begin)
    if p_begin==-1: return None
    s2 = s2[:p_end].replace(begin, '', count=1)

    return p_begin, p_end, s[p_begin:p_end]

s2 = s
res = find_tag(s2, 'b')
while res is not None:
    
    b, e, sub = 
    s2 = s.replace(sub, 'x'*len(sub))


(2, 26, '<b>defxxxxxxxxxxxxxA<b/>')

In [ ]:
def remove_tag(s: str, tag='b'):
    s